# Jira Issue Wrangler

Map-1 issue: https://ubuntu-policy.atlassian.net/browse/MAP-1

Here is the Kanban Board to edit the data: https://ubuntu-policy.atlassian.net/secure/RapidBoard.jspa?rapidView=8&projectKey=MAP

In [20]:
import json

from jira import JIRA
from osbot_jira.api.jira_server.API_Jira import API_Jira


class Issue():
    
    def __init__(self, key, summary, status, related_issues):
        self.key = key
        self.summary = summary
        self.status = status
        self.related_issues = related_issues


class JiraIssueWrangler():
    
    def __init__(self, issue_id):
        self.issue_id = issue_id
        self.api_jira = API_Jira()
        self.api_jira._jira = JIRA(server='https://ubuntu-policy.atlassian.net/')
        
    def _create_issue(self, response):
        return Issue(
            response['Key'],
            response['Summary'],
            response['Status'],
            self._get_related_issues(response)
        )
    
    def _get_related_issues(self, response):
        related_issues = []
        for key, issue_links in response['Issue Links'].items():
            for issue_link in issue_links:
                if issue_link['Link Type'] == 'needs':
                    related_issue_response = self.api_jira.issue(key)
                    related_issues.append(self._create_issue(related_issue_response))

        return related_issues
                    
    def wrangle(self):
        base_issue_response = self.api_jira.issue(self.issue_id)
        issue = self._create_issue(base_issue_response)
        return issue


def fetch_jira_issue(issue_id):
    issue_wrangler = JiraIssueWrangler('Map-1')
    issue = issue_wrangler.wrangle()
    print(json.dumps(
        issue,
        default=lambda o: o.__dict__,
        sort_keys=False,
        indent=2
    ))
    
    
fetch_jira_issue('Map-1')


{
  "key": "MAP-1",
  "summary": "Cup of Tea",
  "status": "Product",
  "related_issues": [
    {
      "key": "MAP-2",
      "summary": "Cup",
      "status": "Product",
      "related_issues": []
    },
    {
      "key": "MAP-3",
      "summary": "Tea",
      "status": "Product",
      "related_issues": [
        {
          "key": "MAP-9",
          "summary": "Farmer",
          "status": "Commodity",
          "related_issues": [
            {
              "key": "MAP-7",
              "summary": "Power",
              "status": "Commodity",
              "related_issues": []
            }
          ]
        }
      ]
    },
    {
      "key": "MAP-4",
      "summary": "Hot Water",
      "status": "Commodity",
      "related_issues": [
        {
          "key": "MAP-6",
          "summary": "Kettle",
          "status": "Custom Built",
          "related_issues": [
            {
              "key": "MAP-7",
              "summary": "Power",
              "status": "Commodity"